# EMToolKit Dashboard Example

## This software shows the user how to use the dashboard

### Import Statements

In [1]:
# Standard library imports
import importlib
import os.path
import pickle
import time
from sys import path

# Third-party library imports
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import matplotlib
from sunpy.map import Map
from sunpy.net import Fido, attrs as a, vso
from sunpy.time import TimeRange
from ndcube import NDCube, NDCubeSequence, NDCollection
from astropy import wcs

# EMToolKit specific imports
import EMToolKit
import EMToolKit.visualization
import EMToolKit.EMToolKit as emtk
from EMToolKit.visualization.dashboard import dashboard_object
from EMToolKit.instruments.aia import load_from_paths, aia_wrapper
from EMToolKit.algorithms.sparse_em_wrapper import sparse_em_wrapper
from EMToolKit.algorithms.simple_reg_dem_wrapper import simple_reg_dem_wrapper

# Reloading modules (if necessary)
importlib.reload(EMToolKit)
importlib.reload(EMToolKit.visualization)

# Jupyter/IPython specific commands
%matplotlib widget
# matplotlib.use("qt5Agg")
# jupyter nbextension enable --py widgetsnbextension

# Configuration settings
plt.rcParams.update({'font.size': 18, 'figure.figsize': [27, 15], 'image.origin': 'lower'})
np.set_printoptions(linewidth=128)

# Setting the base path
base_path = ""
emtk_path = base_path + 'code/'
path.append(emtk_path)
os.makedirs(emtk_path, exist_ok=True)

# Display HTML for notebook width
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

# Initialization
em_collection = 0

### Download the Imagery

In [2]:
xlen = None
ylen = None
xl = None
yl = None

sdo_data_dir = base_path+"data/20220330/" # Place to put data files. You'll need to change it or create these subdirectories

paths = [sdo_data_dir + path for path in os.listdir(sdo_data_dir)
         if (os.path.isfile(os.path.join(sdo_data_dir, path)) and ".fits" in path)]

paths = sorted(paths)
paths = [paths[5], paths[0], paths[1], paths[2], paths[3], paths[4]]

print(paths)

['data/20220330/aia_lev1_94a_2022_03_30t12_04_11_12z_image_lev1.fits', 'data/20220330/aia_lev1_131a_2022_03_30t12_04_06_62z_image_lev1.fits', 'data/20220330/aia_lev1_171a_2022_03_30t12_04_09_35z_image_lev1.fits', 'data/20220330/aia_lev1_193a_2022_03_30t12_04_04_84z_image_lev1.fits', 'data/20220330/aia_lev1_211a_2022_03_30t12_04_09_63z_image_lev1.fits', 'data/20220330/aia_lev1_335a_2022_03_30t12_04_00_62z_image_lev1.fits']


In [3]:
# Load the AIA data:
print(paths)
maps = load_from_paths(paths,xl=xl,yl=yl,dx=xlen,dy=ylen)

temperatures = np.arange(5.5, 7.5, 0.1)

[maps,errs,trlogts,tresps] = aia_wrapper(maps, temperature_array=np.array(temperatures))

# Make data sequence from AIA data:
datasequence = emtk.em_data(maps,errs,trlogts,tresps)

# Create em_collection with AIA data:
em_collection = emtk.em_collection(datasequence)


['data/20220330/aia_lev1_94a_2022_03_30t12_04_11_12z_image_lev1.fits', 'data/20220330/aia_lev1_131a_2022_03_30t12_04_06_62z_image_lev1.fits', 'data/20220330/aia_lev1_171a_2022_03_30t12_04_09_35z_image_lev1.fits', 'data/20220330/aia_lev1_193a_2022_03_30t12_04_04_84z_image_lev1.fits', 'data/20220330/aia_lev1_211a_2022_03_30t12_04_09_63z_image_lev1.fits', 'data/20220330/aia_lev1_335a_2022_03_30t12_04_00_62z_image_lev1.fits']


### Compute the Differential Emission Measures

In [4]:
# Compute the DEM for Cheung et al. sparse EM method:
# Make DEM sequence from the DEM and add it to the collection:
pk_file = f'{sdo_data_dir}/sparse_em_demsequence.pkl'

if os.path.exists(pk_file) and False:
    with open(pk_file, 'rb') as file:
       (sparse_em_demsequence, spars_out) = pickle.load(file)
else:
    tstart=time.time()
    spars_out = sparse_em_wrapper(datasequence)
    sparse_em_demsequence = emtk.dem_model(*spars_out, sparse_em_wrapper)
    print('Sparse method took',time.time()-tstart)
    with open(pk_file, 'wb') as file:
        pickle.dump((sparse_em_demsequence, spars_out), file)

spars_coeffs, spars_logts, spars_bases, spars_coords, spars_algorithm = spars_out
em_collection.add_model(sparse_em_demsequence)

IndexError: list index out of range

In [5]:
# Compute the DEM for Plowman & Caspi simple_reg_dem method:
# Make DEM sequence from the DEM and add it to the collection:
pk_file = f'{sdo_data_dir}/simple_reg_demsequence.pkl'
# if(('simple_reg_demsequence' in locals()) == False):
recalc = False
if os.path.exists(pk_file):
    with open(pk_file, 'rb') as file:
        (simple_reg_demsequence, simpl_out) = pickle.load(file)
else:
    tstart=time.time()
    simpl_out = simple_reg_dem_wrapper(datasequence)
    print('Simple method took',time.time()-tstart)
    simple_reg_demsequence = emtk.dem_model(*simpl_out, simple_reg_dem_wrapper)
    with open(pk_file, 'wb') as file:
        pickle.dump((simple_reg_demsequence, simpl_out), file)

em_collection.add_model(simple_reg_demsequence)
simpl_coeffs, simpl_logts, simpl_bases, simpl_coords, simpl_algorithm = simpl_out

In [6]:
simpl_synthdata = em_collection.synthesize_data(trlogts,tresps,algorithm=simpl_algorithm)
spars_synthdata = em_collection.synthesize_data(trlogts,tresps,algorithm=spars_algorithm)

ptlogt,dem=em_collection.compute_dem_all(logt=np.arange(5.5,7.5,0.1),algorithm=simpl_algorithm)
ptlogt2,dem2=em_collection.compute_dem_all(logt=np.arange(5.5,7.5,0.1),algorithm=spars_algorithm)

# # save dem and dem2 to pkl files
# with open(f'{sdo_data_dir}/simple_output.pkl', 'wb') as file:
#     pickle.dump((ptlogt,dem), file)

# with open(f'{sdo_data_dir}/sparse_output.pkl', 'wb') as file:
#     pickle.dump((ptlogt2,dem2), file)

In [ ]:
# compare dem outputs
import pickle

with open("idl_output.pkl", "rb") as f:
    idl = pickle.load(f)  # Load the single object

idl_dem = idl['EMCUBE']

cpy = np.copy(idl_dem)

for i in range(20):
    slice1 = np.log10(idl_dem[i, :, :])
    slice2 = np.log10(dem[:, :, i])
    slice3 = np.log10(dem2[:, :, i])

    fig, axs = plt.subplots(1, 3)
    im = axs[0].imshow(slice1, cmap='inferno', vmin=20, vmax=27)
    axs[0].set(title='IDL')
    plt.colorbar(im, ax=axs[0])

    im = axs[1].imshow(slice2, cmap='inferno', vmin=20, vmax=27)
    axs[1].set(title='Python (Simple)')
    plt.colorbar(im, ax=axs[1])

    im = axs[2].imshow(slice3, cmap='inferno', vmin=20, vmax=27)
    axs[2].set(title='Python (Sparse)')
    plt.colorbar(im, ax=axs[2])

    os.makedirs('./dem_comparisons/', exist_ok=True)
    plt.savefig(f'./dem_comparisons/dem_comparison_{i}.png')
    plt.close()